In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-10 23:08:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  66.7KB/s    in 9.7s    

2025-09-10 23:08:16 (112 KB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r') as f:
    text = f.read()

In [4]:
print(f"length of dataset in characters: {len(text)}")

length of dataset in characters: 1115394


In [5]:
print(text[:1000])  # first 1000 characters")

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars[::-1]))
print(vocab_size)

all the unique characters: zyxwvutsrqponmlkjihgfedcbaZYXWVUTSRQPONMLKJIHGFEDCBA?;:3.-,'&$! 

65


In [7]:
# Strategy to tokenize the text at character level
# Create a mapping from characters to integers and vice versa
stoi = { ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]  # string to integer
decode = lambda l: ''.join([itos[i] for i in l])  # integer to string

print(encode("hii there, Mahata"))
print(decode(encode("hii there, Mahata")))

[46, 47, 47, 1, 58, 46, 43, 56, 43, 6, 1, 25, 39, 46, 39, 58, 39]
hii there, Mahata


In [8]:
# Encoding the entire text dataset and store it into a torch tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])  # first 1000 characters encoded as integers

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Train-test split
n = int(0.9*len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]


In [10]:
# We only work with chunks of the datasets of a fixed size
block_size = 8  # context length: how many characters do we take to predict the
train_data[:block_size+1]  # we will use the first block_size characters to predict the last character

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]  # first block_size characters
y = train_data[1:block_size+1]  # next block_size characters
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [18] the target: 47
when input is [18, 47] the target: 56
when input is [18, 47, 56] the target: 57
when input is [18, 47, 56, 57] the target: 58
when input is [18, 47, 56, 57, 58] the target: 1
when input is [18, 47, 56, 57, 58, 1] the target: 15
when input is [18, 47, 56, 57, 58, 1, 15] the target: 47
when input is [18, 47, 56, 57, 58, 1, 15, 47] the target: 58


In [14]:
torch.manual_seed(1338)
batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the maximum context length for predictions?   

def get_batch(split):
    # generate a small batch data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print(f"inputs: {xb.shape} ", xb)
print(f"targets: {yb.shape} ", yb)


print('------------------------------')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs: torch.Size([4, 8])  tensor([[10,  0, 32, 46, 53, 59,  1, 57],
        [56,  8,  0, 21, 44,  1, 35, 39],
        [ 0, 17, 42, 61, 39, 56, 42,  1],
        [42, 45, 47, 52, 45, 57,  1, 39]])
targets: torch.Size([4, 8])  tensor([[ 0, 32, 46, 53, 59,  1, 57, 43],
        [ 8,  0, 21, 44,  1, 35, 39, 56],
        [17, 42, 61, 39, 56, 42,  1, 39],
        [45, 47, 52, 45, 57,  1, 39, 52]])
------------------------------
when input is [10] the target: 0
when input is [10, 0] the target: 32
when input is [10, 0, 32] the target: 46
when input is [10, 0, 32, 46] the target: 53
when input is [10, 0, 32, 46, 53] the target: 59
when input is [10, 0, 32, 46, 53, 59] the target: 1
when input is [10, 0, 32, 46, 53, 59, 1] the target: 57
when input is [10, 0, 32, 46, 53, 59, 1, 57] the target: 43
when input is [56] the target: 8
when input is [56, 8] the target: 0
when input is [56, 8, 0] the target: 21
when input is [56, 8, 0, 21] the target: 44
when input is [56, 8, 0, 21, 44] the target: 1
w

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1330)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)  # (B,T,C)  batch, time, channel (v
print(loss)
    


torch.Size([32, 65])
tensor(4.4897, grad_fn=<NllLossBackward0>)
